
@Author:Vijay Kumar M N<br>
@Date: 2024-10-25<br>
@Last Modified by:Vijay Kumar M N<br>
@Last Modified: 2024-10-25<br>
@Title :Python Program to interact with dynamodb and import and export to the s3 bucket<br>

In [14]:
import boto3
import json
from decimal import Decimal
import os

# Set up AWS clients
s3_client = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')

def decimal_to_float(obj):
    """
    Recursively converts Decimal types to float in a nested JSON structure.
    """
    if isinstance(obj, list):
        return [decimal_to_float(i) for i in obj]
    elif isinstance(obj, dict):
        return {k: decimal_to_float(v) for k, v in obj.items()}
    elif isinstance(obj, Decimal):
        return float(obj)
    else:
        return obj

def export_dynamodb_to_s3(table_name, bucket_name, s3_key):
    """
    Description:
        Exports data from a DynamoDB table to a JSON file, 
        then uploads it to an S3 bucket.
    Parameters:
        table_name:
        bucket_name:
        s3_key

    returns:
        None
    """
    # Get DynamoDB table
    table = dynamodb.Table(table_name)

    # Scan table and retrieve all items
    response = table.scan()
    items = response['Items']

    # Convert all Decimals to floats for JSON serialization
    items = decimal_to_float(items)

    # Save data to a JSON file
    json_file = "/tmp/dynamodb_data.json"
    with open(json_file, 'w') as file:
        json.dump(items, file, indent=4)

    # Upload JSON file to S3
    s3_client.upload_file(json_file, bucket_name, s3_key)
    os.remove(json_file)  # Clean up local file after upload

    print(f"Data exported to S3 bucket '{bucket_name}' with key '{s3_key}'")

def float_to_decimal(obj):
    """
    Recursively converts float types to Decimal in a nested JSON structure.
    """
    if isinstance(obj, list):
        return [float_to_decimal(i) for i in obj]
    elif isinstance(obj, dict):
        return {k: float_to_decimal(v) for k, v in obj.items()}
    elif isinstance(obj, float):
        return Decimal(str(obj))
    else:
        return obj

def import_dynamodb_from_s3(table_name, bucket_name, s3_key):
    """
    Imports data from an S3 JSON file into a DynamoDB table and saves a local copy.
    """
    # Define the local JSON file path
    json_file = "/tmp/dynamodb_data.json"

    # Download the file from S3
    s3_client.download_file(bucket_name, s3_key, json_file)

    # Load data from JSON file
    with open(json_file, 'r') as file:
        items = json.load(file)

    # Convert floats to Decimals
    items = float_to_decimal(items)

    # Insert items into DynamoDB
    table = dynamodb.Table(table_name)
    with table.batch_writer() as batch:
        for item in items:
            batch.put_item(Item=item)

    # Save a copy of the downloaded data to a local file
    local_copy_file = f"./local_imported_data.json"  # Define a path for the local copy
    with open(local_copy_file, 'w') as file:
        # Convert items to floats for JSON serialization
        items_for_local_copy = decimal_to_float(items)
        json.dump(items_for_local_copy, file, indent=4)

    os.remove(json_file)  # Clean up the temporary file after import
    print(f"Data imported to DynamoDB table '{table_name}' from S3 bucket '{bucket_name}'")
    print(f"A local copy of the imported data is saved as '{local_copy_file}'")

def main():
    # Set table and S3 bucket details
    table_name = "Movies"
    bucket_name = "dynamodb-vk"
    s3_key = "dynamodb_exports/dynamodb_data.json"

    while True:
        # Ask the user whether they want to export or import data
        print("\n1. Export data from DynamoDB to S3")
        print("2. Import data from S3 to DynamoDB")
        print("3. Exit")
        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == "1":
            export_dynamodb_to_s3(table_name, bucket_name, s3_key)
        elif choice == "2":
            import_dynamodb_from_s3(table_name, bucket_name, s3_key)
        elif choice == "3":
            print("Exiting from the program...")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()



1. Export data from DynamoDB to S3
2. Import data from S3 to DynamoDB
3. Exit
Invalid choice. Please enter 1, 2, or 3.

1. Export data from DynamoDB to S3
2. Import data from S3 to DynamoDB
3. Exit
Data imported to DynamoDB table 'Movies' from S3 bucket 'dynamodb-vk'
A local copy of the imported data is saved as './local_imported_data.json'

1. Export data from DynamoDB to S3
2. Import data from S3 to DynamoDB
3. Exit
Exiting from the program...
